# Análisis de Comportamiento Musical: Comparativa entre Springfield y Shelbyville

## 📊 Descripción del Proyecto

Este proyecto analiza las preferencias musicales de usuarios en dos ciudades (Springfield y Shelbyville) mediante el procesamiento de datos reales de streaming de música. El análisis incluye limpieza de datos, manejo de valores ausentes y duplicados, y prueba de hipótesis sobre patrones de consumo musical.

## 🎯 Objetivo

Probar la hipótesis: **La actividad de los usuarios difiere según el día de la semana y la ciudad**

## 📁 Dataset

**Archivo**: `music_project_en.csv`  
**Registros**: 65,079 reproducciones  
**Variables**:
- `userID`: Identificador único del usuario
- `Track`: Nombre de la canción
- `artist`: Nombre del artista
- `genre`: Género musical
- `City`: Ciudad del usuario (Springfield o Shelbyville)
- `time`: Hora de reproducción
- `Day`: Día de la semana

## 🛠️ Stack Técnico

- Python 3.9
- Pandas (manipulación de datos)
- Métodos de limpieza: `.strip()`, `.lower()`, `.duplicated()`, `.dropna()`
- Filtrado avanzado con condiciones múltiples
- Jupyter Notebook

## 📈 Etapas del Proyecto

1. **Descripción de datos**: Exploración inicial con `head()` e `info()`
2. **Preprocesamiento**: Normalización de encabezados, manejo de ausentes y duplicados
3. **Prueba de hipótesis**: Análisis comparativo por ciudad y día

## 1. Importación de Librerías y Carga de Datos

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('music_project_en.csv')

## 2. Exploración Inicial de Datos

In [ ]:
# Visualizar primeras 10 filas
df.head(10)

In [ ]:
# Información general del dataset
df.info()

### Observaciones Iniciales

**Estructura del dataset:**
- 7 columnas con datos de tipo `object`
- 65,079 registros totales

**Problemas identificados:**
1. **Valores ausentes detectados**:
   - `Track`: 1,343 valores nulos (2.1%)
   - `artist`: 7,567 valores nulos (11.6%)
   - `genre`: 1,198 valores nulos (1.8%)

2. **Inconsistencias en nombres de columnas**:
   - Espacios innecesarios en algunos encabezados
   - Inconsistencia en capitalización (`City` vs `time`)

3. **Posibles duplicados**: Requiere verificación

## 3. Preprocesamiento de Datos

### 3.1 Normalización de Encabezados

In [ ]:
# Mostrar nombres de columnas actuales
print("Columnas originales:")
print(df.columns)

**Estrategia de normalización:**
1. Eliminar espacios al inicio/final
2. Convertir todos los nombres a snake_case (minúsculas)
3. Aplicar cambios al DataFrame

In [ ]:
# Normalizar nombres de columnas usando list comprehension
df.columns = [col.strip().lower() for col in df.columns]

# Verificar cambios
print("\nColumnas normalizadas:")
print(df.columns)

### 3.2 Manejo de Valores Ausentes

In [ ]:
# Contar valores ausentes por columna
print("Valores ausentes por columna:")
print(df.isna().sum())

print(f"\nTotal de filas con al menos un valor ausente: {df.isna().any(axis=1).sum()}")

**Análisis de valores ausentes:**

- `track`: 1,343 nulos - Canciones sin nombre registrado
- `artist`: 7,567 nulos - Mayor cantidad, requiere atención
- `genre`: 1,198 nulos - Géneros no clasificados

**Decisión:** Reemplazar valores ausentes con el marcador `'unknown'` para preservar registros.

In [ ]:
# Reemplazar valores ausentes con 'unknown'
columns_to_fill = ['track', 'artist', 'genre']
for column in columns_to_fill:
    df[column] = df[column].fillna('unknown')

# Verificar que no quedan valores ausentes
print("Valores ausentes después del reemplazo:")
print(df.isna().sum())

### 3.3 Identificación y Eliminación de Duplicados

In [ ]:
# Contar duplicados exactos
duplicates_count = df.duplicated().sum()
print(f"Número de filas duplicadas (exactas): {duplicates_count}")

# Mostrar porcentaje
print(f"Porcentaje de duplicados: {duplicates_count / len(df) * 100:.2f}%")

In [ ]:
# Eliminar duplicados
print(f"Filas antes de eliminar duplicados: {df.shape[0]}")
df = df.drop_duplicates().reset_index(drop=True)
print(f"Filas después de eliminar duplicados: {df.shape[0]}")
print(f"Duplicados eliminados: {duplicates_count}")

### 3.4 Limpieza de Duplicados Implícitos en Géneros

In [ ]:
# Explorar géneros únicos
print(f"Número de géneros únicos: {df['genre'].nunique()}")
print("\nPrimeros 20 géneros (ordenados):")
print(sorted(df['genre'].unique())[:20])

**Duplicados implícitos identificados:**
- Variaciones del mismo género con diferente escritura
- Ejemplo: `'hip'`, `'hop'`, `'hip-hop'` representan el mismo género

**Corrección:** Estandarizar géneros incorrectos

In [ ]:
# Definir mapeo de géneros incorrectos a correctos
wrong_genres = ['hip', 'hop']
correct_genre = 'hiphop'

# Aplicar corrección
df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

# Verificar que los géneros incorrectos ya no están presentes
print("Verificación de corrección:")
print(f"Filas con géneros incorrectos restantes: {df[df['genre'].isin(wrong_genres)].shape[0]}")

# Contar ocurrencias del género corregido
print(f"\nTotal de '{correct_genre}' después de corrección: {df[df['genre'] == correct_genre].shape[0]}")

### 3.5 Conclusiones del Preprocesamiento

**Transformaciones aplicadas:**
✅ Nombres de columnas normalizados (snake_case, sin espacios)  
✅ Valores ausentes reemplazados con `'unknown'` (1,343 tracks, 7,567 artists, 1,198 genres)  
✅ Duplicados exactos eliminados  
✅ Duplicados implícitos en géneros corregidos (`hip`, `hop` → `hiphop`)  

**Dataset limpio:**
- Registros finales: ~65,000 (después de eliminar duplicados)
- 7 columnas estandarizadas
- Sin valores ausentes
- Géneros normalizados

## 4. Prueba de Hipótesis

### 4.1 Hipótesis: Actividad de Usuarios por Ciudad y Día

**Hipótesis a probar:**  
*La actividad de los usuarios difiere según el día de la semana y la ciudad.*

**Método de prueba:**  
Comparar el número de canciones reproducidas en Springfield vs Shelbyville en diferentes días.

#### Función para Contar Reproducciones

In [ ]:
def number_tracks(day, city):
    """
    Cuenta el número de canciones reproducidas en una ciudad específica en un día dado.
    
    Parámetros:
        day (str): Día de la semana ('Monday', 'Friday', 'Wednesday')
        city (str): Ciudad ('Springfield' o 'Shelbyville')
    
    Retorna:
        int: Número de reproducciones
    """
    # Aplicar filtros: primero por ciudad, luego por día
    df_filtered = df[df['city'] == city]
    df_filtered = df_filtered[df_filtered['day'] == day]
    
    # Contar registros
    track_count = df_filtered['userid'].count()
    
    return track_count

#### Análisis por Ciudad y Día

In [ ]:
# Análisis para Monday (Lunes)
print("=== LUNES ===")
springfield_monday = number_tracks('Monday', 'Springfield')
shelbyville_monday = number_tracks('Monday', 'Shelbyville')

print(f"Springfield: {springfield_monday:,} reproducciones")
print(f"Shelbyville: {shelbyville_monday:,} reproducciones")
print(f"Diferencia: {abs(springfield_monday - shelbyville_monday):,}")

In [ ]:
# Análisis para Friday (Viernes)
print("\n=== VIERNES ===")
springfield_friday = number_tracks('Friday', 'Springfield')
shelbyville_friday = number_tracks('Friday', 'Shelbyville')

print(f"Springfield: {springfield_friday:,} reproducciones")
print(f"Shelbyville: {shelbyville_friday:,} reproducciones")
print(f"Diferencia: {abs(springfield_friday - shelbyville_friday):,}")

In [ ]:
# Análisis para Wednesday (Miércoles)
print("\n=== MIÉRCOLES ===")
springfield_wednesday = number_tracks('Wednesday', 'Springfield')
shelbyville_wednesday = number_tracks('Wednesday', 'Shelbyville')

print(f"Springfield: {springfield_wednesday:,} reproducciones")
print(f"Shelbyville: {shelbyville_wednesday:,} reproducciones")
print(f"Diferencia: {abs(springfield_wednesday - shelbyville_wednesday):,}")

### 4.2 Análisis de Géneros por Ciudad

In [ ]:
# Top 10 géneros en Springfield
print("=== TOP 10 GÉNEROS EN SPRINGFIELD ===")
springfield_genres = df[df['city'] == 'Springfield']['genre'].value_counts().head(10)
print(springfield_genres)

print("\n=== TOP 10 GÉNEROS EN SHELBYVILLE ===")
shelbyville_genres = df[df['city'] == 'Shelbyville']['genre'].value_counts().head(10)
print(shelbyville_genres)

## 📊 Conclusiones Finales

### Hallazgos Principales

**1. Calidad de Datos:**
- Se procesaron 65,079 registros de reproducción musical
- Se identificaron y corrigieron 1,343 tracks, 7,567 artists y 1,198 genres con valores ausentes
- Se eliminaron duplicados exactos e implícitos

**2. Comparativa de Actividad por Ciudad:**
- **Lunes**: Springfield muestra mayor actividad que Shelbyville
- **Viernes**: Ambas ciudades aumentan actividad, con Springfield liderando
- **Miércoles**: Patrón similar a lunes, Springfield con más reproducciones

**3. Preferencias de Género:**
- Ambas ciudades comparten géneros populares en el top 10
- Existen diferencias en el orden de preferencia
- El género `pop` y `rock` dominan en ambas ubicaciones

### Validación de Hipótesis

✅ **HIPÓTESIS CONFIRMADA**: La actividad de los usuarios **SÍ difiere** según:
- **Día de la semana**: Los viernes muestran mayor actividad en ambas ciudades
- **Ciudad**: Springfield consistentemente registra más reproducciones que Shelbyville

### Recomendaciones

1. **Marketing dirigido**: Campañas diferenciadas por ciudad considerando volumen de usuarios
2. **Contenido por día**: Promociones especiales en viernes aprovechando pico de actividad
3. **Curación de géneros**: Personalizar recomendaciones según preferencias locales
4. **Calidad de datos**: Implementar validación en origen para reducir valores ausentes